In [1]:
import TimeTagger
import numpy as np
import asyncio
import plotly.graph_objs as go
from ipywidgets import Button


In [2]:
# create time tagger object (link to device) and get the active channels
tagger = TimeTagger.createTimeTagger()
input_channels = tagger.getChannelList(TimeTagger.ChannelEdge.Rising)
for ch in input_channels:
    tagger.setTriggerLevel(ch, 0.5)

# get every combination of 2 channels
groups = [(3, 4), (3, 1), (2, 4), (2, 1)]

# 30ns coincidence window to match qutools (coincidence window is in ps)
coincidenceWindowSI = 30e-9 # seconds
coincidences_vchannels = TimeTagger.Coincidences(tagger, groups, coincidenceWindow=coincidenceWindowSI * 1e12)
groups

[(3, 4), (3, 1), (2, 4), (2, 1)]

In [3]:
binwidthSI = 0.1
# convert binwidth to ps
traces = []
for vch in [1,2,3,4]: #coincidences_vchannels.getChannels():
    counter = TimeTagger.Counter(tagger=tagger, channels=[vch], binwidth=binwidthSI * 1e12, n_values=1000)
    traces.append(counter)

# Init figure
fig_trace = go.FigureWidget()

# add scatter for each virtual channel
for trace, group in zip(traces, groups):
    fig_trace.add_scatter(x=trace.getIndex(), y=trace.getData()[0], name=f"|{'R' if group[0] == 3 else 'T'}, {'R' if group[1] == 4 else 'T'}>")

async def update_trace():
    currentMax = 0
    refreshPeriod = binwidthSI if binwidthSI > 0.05 else 0.05
    while True:
        # Batch updates for all traces
        with fig_trace.batch_update():
            for i, trace in enumerate(traces):
                y_data = trace.getData()[0] / binwidthSI

                ymax = np.max(y_data)
                if ymax > currentMax:
                    currentMax = 1 * ymax
                    fig_trace.update_layout(yaxis=dict(range=[0, ymax]))

                fig_trace.data[i].y = y_data

        # do not update faster than binwidth
        await asyncio.sleep(refreshPeriod) 

try:
    task_trace.cancel()
except:
    pass
loop = asyncio.get_event_loop()
task_trace = loop.create_task(update_trace())

button_trace = Button(description="stop")
button_trace.on_click(lambda a: task_trace.cancel())

display(fig_trace, button_trace)

FigureWidget({
    'data': [{'name': '|R, R>',
              'type': 'scatter',
              'uid': 'e53065b5-7555-4f08-828a-e1936514b4e6',
              'x': array([             0,   100000000000,   200000000000, ..., 99700000000000,
                          99800000000000, 99900000000000], dtype=int64),
              'y': array([   0,    0,    0, ..., 2217, 2190, 2201])},
             {'name': '|R, T>',
              'type': 'scatter',
              'uid': 'bc7728a2-a249-4b86-8f49-6a2d408a3d35',
              'x': array([             0,   100000000000,   200000000000, ..., 99700000000000,
                          99800000000000, 99900000000000], dtype=int64),
              'y': array([   0,    0,    0, ..., 2204, 2248, 2183])},
             {'name': '|T, R>',
              'type': 'scatter',
              'uid': '492e7de9-3a32-4176-a008-4dff57475c51',
              'x': array([             0,   100000000000,   200000000000, ..., 99700000000000,
                          9980000000

Button(description='stop', style=ButtonStyle())